In [ ]:
!pip install import_ipynb
!pip install tensorflow-datasets

In [ ]:
 # .py로 매번 수정하기 귀찮아서 ipynb 파일로 import가능하게 만드는 라이브러리
import import_ipynb 

import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import models


GoogLeNet = models.InceptionNet

In [17]:
#하이퍼 파라미터
batch_size = 128
num_epoch = 2
train_accuracies = []
test_accuracies = []


def convert_types(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255
    return image, label

# 데이터 생성
dataset, info = tfds.load('fashion_mnist', as_supervised = True, with_info = True)
dataset_test, dataset_train = dataset['test'], dataset['train']

dataset_train = dataset_train.map(convert_types).shuffle(10000).batch(batch_size)
dataset_test = dataset_test.map(convert_types).batch(batch_size)

print(dataset_train)
print(info)

datagen = ImageDataGenerator(rotation_range=10, horizontal_flip=True, zoom_range=0.1)

# GoogleNet 객체 생성
model = GoogLeNet((28, 28, 1), 10)
model.build(input_shape = (None, 28, 28, 1))
model.summary()

# Prepare Taining 라벨링

############## #사용자 루프 모델 학습 ######
# model.compile() 기능을 사용자가 직접 수동으로 하는 것
#def compile_by_users():
    
# 입력값에 대해 원핫라벨링한 뒤, 교차 엔트로피에 통과하여 loss를 검사
loss_object = keras.losses.SparseCategoricalCrossentropy()
# 최적화는 adam
optimizer = keras.optimizers.Adam()

train_loss = keras.metrics.Mean(name = 'train_loss')
train_accuracy = keras.metrics.SparseCategoricalAccuracy(name = 'train_accuracy')
test_loss = keras.metrics.Mean(name = 'test_loss')
test_accuracy = keras.metrics.SparseCategoricalAccuracy(name = 'test_accuracy')
    


#compile_by_users()

def train_step(image, label): # 훈련 이미지, 훈련 레이블
    with tf.GradientTape() as tape:
        
        predictions = model(image)
        loss = loss_object(label, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_accuracy(label, predictions)
    
def test_step(image, label): # 훈련 이미지, 훈련 레이블
        
    predictions = model(image)
    loss = loss_object(label, predictions)
    
    test_loss(loss)
    test_accuracy(label, predictions)
    


<PrefetchDataset element_spec=(TensorSpec(shape=(28, 28, 1), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>
tfds.core.DatasetInfo(
    name='fashion_mnist',
    full_name='fashion_mnist/3.0.1',
    description="""
    Fashion-MNIST is a dataset of Zalando's article images consisting of a training set of 60,000 examples and a test set of 10,000 examples. Each example is a 28x28 grayscale image, associated with a label from 10 classes.
    """,
    homepage='https://github.com/zalandoresearch/fashion-mnist',
    data_path='C:\\Users\\sangt\\tensorflow_datasets\\fashion_mnist\\3.0.1',
    download_size=29.45 MiB,
    dataset_size=36.42 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'trai

In [6]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
        printEnd    - Optional  : end character (e.g. "\r", "\r\n") (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
    # Print New Line on Complete
    if iteration == total: 
        print()

In [19]:
for epoch in range(num_epoch):
    i = 0
    for image, label in dataset_train:
        # print(image.shape)
        # print(image)
        for _image, _label in datagen.flow(image, label, batch_size = batch_size):
            #print("image : {}, \n label : {}".format(_image, _label))
            printProgressBar(i + 1, len(dataset_train),  prefix = 'Progress:', suffix = 'Complete', length = 50)
            # Data Augmentabtion된 훈련 데이터를 _image, 훈련 레이블을 _label로 저장
            # Data Augmentabtion이란 데이터셋을 여러 방법으로 변경하여 데이터셋 개수를 늘리는 방법이다.
            # 회전을 시킨다거나, 임의의 범위를 잘라낸다건, 하나의 소르를 가지고 데이터를 우려내는 방법이다.
            train_step(_image, _label) # 모델에 data Augmentation된 훈련 데이터셋을 넣음
            i+=1
            break
    # 훈련 데이터 셋에 대한 학습 종료

    for test_image, test_label in dataset_test:
        test_step(test_image, test_label)
        # 시험 데이터 셋 테스트 종료
        
    train_accuracies.append(train_accuracy.result())
    test_accuracies.append(test_accuracy.result())
    
    template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
    print(template.format(epoch + 1, train_loss.result(), train_accuracy.result() * 100,
                          test_loss.result(), test_accuracy.result() * 100))

(128, 28, 28, 1)
tf.Tensor(
[[[[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.        ]]

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.        ]]

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.        ]]

  ...

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.        ]]

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.        ]]

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.        ]]]


 [[[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.        ]]

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.        ]]

  [[0.        ]
   [0.        ]
   [0.        ]
   ...
   [0.        ]
   [0.        ]
   [0.        ]]

  ...

  [[0.     

KeyboardInterrupt: 